ResNet 50 output sizes of stages

In [5]:
from mmdet.models import ResNet
import torch
res50 = ResNet(depth=50)
res50.eval()
inputs =  torch.rand(1, 3, 224, 224)
level_outputs = res50.forward(inputs)
for level_out in level_outputs:
    print(tuple(level_out.shape))

(1, 256, 56, 56)
(1, 512, 28, 28)
(1, 1024, 14, 14)
(1, 2048, 7, 7)


In [6]:
from torchsummary import summary
summary(res50, (1, 3, 224, 224))

RuntimeError: Failed to run torchsummary. See above stack traces for more details. Executed layers up to: []

### Test inference FeedbackNet

In [1]:
# adjust keys in checkpoint weights file
import torch
checkpoint = torch.load('../../checkpoints/faster_rcnn_r50_fpn_1x_voc0712_20220320_192712-54bef0f3.pth')
state_dict = checkpoint['state_dict']
for key in list(state_dict.keys()):
    state_dict[key.replace("backbone.","backbone.resnet.")] = state_dict.pop(key)
checkpoint['state_dict'] = state_dict
torch.save(checkpoint, '../../checkpoints/faster_rcnn_r50fb_fpn_1x_voc0712_20221208.pth')

In [2]:
from mmdet.apis import init_detector, inference_detector
import mmcv

# Specify the path to model config and checkpoint file
config_file = '../../configs/pascal_voc/faster_rcnn_r50fb_fpn_1x_voc0712.py'
checkpoint_file = '../../checkpoints/faster_rcnn_r50fb_fpn_1x_voc0712_20221208.pth'

# build the model from a config file and a checkpoint file
#model = init_detector(config_file, checkpoint_file, device='cpu')
model = init_detector(config_file, checkpoint_file, device='cpu')

c:\Users\lerch\anaconda3\envs\cv_project\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


load checkpoint from local path: ../../checkpoints/faster_rcnn_r50fb_fpn_1x_voc0712_20221208.pth
The model and loaded state dict do not match exactly

missing keys in source state_dict: backbone.fb_con.conv.weight, backbone.fb_con.conv.bias



In [3]:
# will only load it once
img = mmcv.imread("../../demo/demo.jpg")
result = inference_detector(model, img)
# visualize the results in a new window
model.show_result(img, result, out_file='../../output/result.jpg')

c:\users\lerch\documents\studium\master\wise22\project_cv\code\mmdetectionrobustness\mmdet\datasets\utils.py:66: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(


In [2]:
# Print model's state_dict
# print("Model's state_dict:")
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())

#print(model)
from torchsummary import summary
summary(model)#, (1, 3, 224, 224))

Layer (type:depth-idx)                   Param #
├─FeedbackResNet: 1-1                    --
|    └─ResNet: 2-1                       --
|    |    └─Conv2d: 3-1                  (9,408)
|    |    └─BatchNorm2d: 3-2             (128)
|    |    └─ReLU: 3-3                    --
|    |    └─MaxPool2d: 3-4               --
|    |    └─ResLayer: 3-5                (215,808)
|    |    └─ResLayer: 3-6                1,219,584
|    |    └─ResLayer: 3-7                7,098,368
|    |    └─ResLayer: 3-8                14,964,736
|    └─FBConnection: 2-2                 --
|    |    └─Upsample: 3-9                --
|    |    └─Conv2d: 3-10                 55,299
|    |    └─LocalResponseNorm: 3-11      --
├─FPN: 1-2                               --
|    └─ModuleList: 2-3                   --
|    |    └─ConvModule: 3-12             65,792
|    |    └─ConvModule: 3-13             131,328
|    |    └─ConvModule: 3-14             262,400
|    |    └─ConvModule: 3-15             524,544
|    └─Modu

Layer (type:depth-idx)                   Param #
├─FeedbackResNet: 1-1                    --
|    └─ResNet: 2-1                       --
|    |    └─Conv2d: 3-1                  (9,408)
|    |    └─BatchNorm2d: 3-2             (128)
|    |    └─ReLU: 3-3                    --
|    |    └─MaxPool2d: 3-4               --
|    |    └─ResLayer: 3-5                (215,808)
|    |    └─ResLayer: 3-6                1,219,584
|    |    └─ResLayer: 3-7                7,098,368
|    |    └─ResLayer: 3-8                14,964,736
|    └─FBConnection: 2-2                 --
|    |    └─Upsample: 3-9                --
|    |    └─Conv2d: 3-10                 55,299
|    |    └─LocalResponseNorm: 3-11      --
├─FPN: 1-2                               --
|    └─ModuleList: 2-3                   --
|    |    └─ConvModule: 3-12             65,792
|    |    └─ConvModule: 3-13             131,328
|    |    └─ConvModule: 3-14             262,400
|    |    └─ConvModule: 3-15             524,544
|    └─Modu